In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# a. Load and preprocess dataset
data = pd.read_csv('/Users/akshay/Downloads/demo_dl/datasets1/ECGdataset(Ass4)/ecg_autoencoder_dataset.csv')
data['Amount'] = StandardScaler().fit_transform(data[['Amount']])
data = data.drop(['Time'], axis=1)

# Separate normal transactions (Class = 0) for training
x_train = data[data['Class'] == 0].drop(['Class'], axis=1).values

# Test data (normal + fraud)
x_test = data.drop(['Class'], axis=1).values
y_test = data['Class'].values

# b. Build Autoencoder model
inp = Input((29,))
enc = Dense(16, activation='relu')(inp)
lat = Dense(8, activation='relu')(enc)
dec = Dense(16, activation='relu')(lat)
out = Dense(29, activation='linear')(dec)
model = Model(inp, out)

# c. Compile & Train
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(x_train, x_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# d. Reconstruction on test data
reconstructions = model.predict(x_test)

# e. Calculate reconstruction error
mse = np.mean(np.power(x_test - reconstructions, 2), axis=1)

# f. Set threshold for anomaly detection
threshold = np.percentile(mse, 95)  # top 5% errors = fraud
print("Threshold: ", threshold)

# g. Predict anomalies
y_pred = (mse > threshold).astype(int)

# h. Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraud']))

print("Accuracy: ", accuracy_score(y_test, y_pred))